In [1]:
import math
from multiprocessing import Pool
import os
import sys
import itertools
import time
import tempfile
import pickle

sys.path.append(os.path.abspath(os.pardir))

%matplotlib inline

import keras
import numpy as np
import pandas as pd
import seaborn as sns

# Metrics
from sklearn.metrics import accuracy_score

# Notebook helper methods
from tdparse import notebook_helper
# Models
from tdparse.models.tdlstm import LSTM, TDLSTM, TCLSTM
# Tokenisers
from tdparse.tokenisers import ark_twokenize
# Word Vectors
from tdparse.word_vectors import PreTrained, GloveTwitterVectors
# Get the data
from tdparse.parsers import semeval_14, semeval_15_16, dong, election
from tdparse.data_types import TargetCollection
from tdparse.helper import read_config, full_path
from tdparse.evaluation import evaluation_results

Using TensorFlow backend.


In [2]:
# Load all of the datasets
youtubean_train = semeval_14(full_path(read_config('youtubean_train')))
youtubean_test = semeval_14(full_path(read_config('youtubean_test')))
semeval_14_rest_train = semeval_14(full_path(read_config('semeval_2014_rest_train')))
semeval_14_lap_train = semeval_14(full_path(read_config('semeval_2014_lap_train')))
semeval_14_rest_test = semeval_14(full_path(read_config('semeval_2014_rest_test')))
semeval_14_lap_test = semeval_14(full_path(read_config('semeval_2014_lap_test')))
semeval_15_rest_test = semeval_15_16(full_path(read_config('semeval_2015_rest_test')))
semeval_16_rest_test = semeval_15_16(full_path(read_config('semeval_2016_rest_test')),
                                     sep_16_from_15=True)
dong_train = dong(full_path(read_config('dong_twit_train_data')))
dong_test = dong(full_path(read_config('dong_twit_test_data')))
election_train, election_test = election(full_path(read_config('election_folder_dir')))
product_reviews_train = semeval_14(full_path(read_config('product_train')))
product_reviews_test = semeval_14(full_path(read_config('product_test')))
# Combine semeval 14 resturant train and test
semeval_14_rest_all = TargetCollection.combine_collections(semeval_14_rest_train,
                                                           semeval_14_rest_test)
# Combine semeval 14 resturant all with 15 test
semeval_14_15 = TargetCollection.combine_collections(semeval_14_rest_all,
                                                     semeval_15_rest_test)

dataset_train_test = {'SemEval 14 Laptop' : (semeval_14_lap_train, semeval_14_lap_test),
                      'SemEval 14 Restaurant' : (semeval_14_rest_train, semeval_14_rest_test),
                      'SemEval 16 Restaurant 14 Train' : (semeval_14_rest_train, semeval_16_rest_test),
                      'SemEval 16 Restaurant 14 All' : (semeval_14_rest_all, semeval_16_rest_test),
                      'SemEval 16 Restaurant 15&14' : (semeval_14_15, semeval_16_rest_test),
                      'Dong Twitter' : (dong_train, dong_test),
                      'Election Twitter' : (election_train, election_test),
                      'Product Reviews' : (product_reviews_train, product_reviews_test),
                      'YouTuBean' : (youtubean_train, youtubean_test)
                     }

In [3]:
# Loading the word vectors
glove_twit_200 = GloveTwitterVectors(200)


In [4]:
def half_average_dataset(dataset):
    '''
    :param dataset: A training dataset
    :type dataset: TargetCollection
    :returns: Half the average sentence length of the given dataset
    :rtype: int
    '''
    sentence_lengths = [len(data['text'].split()) for data in dataset.data()]
    return math.ceil(sum(sentence_lengths) / len(sentence_lengths) / 2)

In [5]:
def dataset_predictions(train, test, dataset_name, model, model_params):
    model_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'model zoo'))
    
    print('{} {}'.format(dataset_name, model_params))
    model = model(**model_params)

    data_train = train.data_dict()
    y_train = train.sentiment_data()
    data_test = test.data_dict()
    y_test = test.sentiment_data()
    
    # Fits the model
    model.fit(data_train, y_train, validation_size=0.3, verbose=1,
              reproducible=True, patience=10, epochs=300, org_initialisers=True)
    # Saves the model to the model zoo
    model_folder_join = lambda file_name: os.path.join(model_dir, file_name)
    model_arch_fp = model_folder_join('{} {} architecture'.format(model, dataset_name))
    model_weights_fp = model_folder_join('{} {} weights'.format(model, dataset_name))
    model.save_model(model_arch_fp, model_weights_fp, verbose=1)
    
    # Predicts on the test data
    predicted_values = model.predict(data_test)
    result_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'results', 'TDLstm'))
    result_folder_join = lambda file_name: os.path.join(result_folder, file_name)
    model_results_file = result_folder_join('{}.tsv'.format(model))
    predicted_values_cats =  model.prediction_to_cats(y_test, predicted_values)
    # Evaluates the predictions and save the results
    return evaluation_results(predicted_values_cats, test, dataset_name, 
                              file_name=model_results_file, 
                              save_raw_data=True, re_write=True)

# Mass Evaluation of the LSTM model

In [ ]:
for dataset_name, train_test in dataset_train_test.items():
    train, test = train_test
    model_params = {'tokeniser' : ark_twokenize, 'embeddings' : glove_twit_200,
                    'lower' : True, 'pad_size' : -1}
    dataset_predictions(train, test, dataset_name, LSTM, model_params)

SemEval 14 Laptop {'tokeniser': <function ark_twokenize at 0x7f8ed20ae0d0>, 'embeddings': glove twitter 200d, 'lower': True, 'pad_size': -1}
Train on 1619 samples, validate on 694 samples
Epoch 1/300
1619/1619 [==============================] - 21s 13ms/step - loss: 1.0924 - acc: 0.4182 - val_loss: 1.0861 - val_acc: 0.4265
Epoch 2/300
1619/1619 [==============================] - 21s 13ms/step - loss: 1.0815 - acc: 0.4268 - val_loss: 1.0770 - val_acc: 0.4265
Epoch 3/300
1619/1619 [==============================] - 21s 13ms/step - loss: 1.0737 - acc: 0.4268 - val_loss: 1.0704 - val_acc: 0.4265
Epoch 4/300
1619/1619 [==============================] - 20s 13ms/step - loss: 1.0681 - acc: 0.4268 - val_loss: 1.0657 - val_acc: 0.4265
Epoch 5/300
1619/1619 [==============================] - 20s 13ms/step - loss: 1.0639 - acc: 0.4268 - val_loss: 1.0621 - val_acc: 0.4265
Epoch 6/300
1619/1619 [==============================] - 20s 13ms/step - loss: 1.0608 - acc: 0.4268 - val_loss: 1.0596 - val_ac

# Mass Evaluation of the TDLSTM model

In [6]:
for dataset_name, train_test in dataset_train_test.items():
    train, test = train_test
    model_params = {'tokeniser' : ark_twokenize, 'embeddings' : glove_twit_200,
                    'lower' : True, 'pad_size' : half_average_dataset(train)}
    dataset_predictions(train, test, dataset_name, TDLSTM, model_params)

SemEval 14 Laptop {'tokeniser': <function ark_twokenize at 0x7f85f53060d0>, 'embeddings': glove twitter 200d, 'lower': True, 'pad_size': 10}
Train on 1619 samples, validate on 694 samples
Epoch 1/300
1619/1619 [==============================] - 8s 5ms/step - loss: 1.0752 - acc: 0.4188 - val_loss: 1.0620 - val_acc: 0.4265
Epoch 2/300
1619/1619 [==============================] - 8s 5ms/step - loss: 1.0568 - acc: 0.4268 - val_loss: 1.0543 - val_acc: 0.4265
Epoch 3/300
1619/1619 [==============================] - 8s 5ms/step - loss: 1.0510 - acc: 0.4268 - val_loss: 1.0503 - val_acc: 0.4265
Epoch 4/300
1619/1619 [==============================] - 9s 5ms/step - loss: 1.0479 - acc: 0.4274 - val_loss: 1.0479 - val_acc: 0.4467
Epoch 5/300
1619/1619 [==============================] - 8s 5ms/step - loss: 1.0455 - acc: 0.4441 - val_loss: 1.0454 - val_acc: 0.4366
Epoch 6/300
1619/1619 [==============================] - 8s 5ms/step - loss: 1.0428 - acc: 0.4373 - val_loss: 1.0430 - val_acc: 0.4553
Ep

Epoch 60/300
1619/1619 [==============================] - 8s 5ms/step - loss: 0.7663 - acc: 0.6671 - val_loss: 0.8075 - val_acc: 0.6369
Epoch 61/300
1619/1619 [==============================] - 8s 5ms/step - loss: 0.7639 - acc: 0.6695 - val_loss: 0.8057 - val_acc: 0.6340
Epoch 62/300
1619/1619 [==============================] - 5s 3ms/step - loss: 0.7620 - acc: 0.6733 - val_loss: 0.8357 - val_acc: 0.6311
Epoch 63/300
1619/1619 [==============================] - 8s 5ms/step - loss: 0.7533 - acc: 0.6782 - val_loss: 0.7960 - val_acc: 0.6441
Epoch 64/300
1619/1619 [==============================] - 5s 3ms/step - loss: 0.7476 - acc: 0.6875 - val_loss: 0.8129 - val_acc: 0.6326
Epoch 65/300
1619/1619 [==============================] - 8s 5ms/step - loss: 0.7475 - acc: 0.6788 - val_loss: 0.7929 - val_acc: 0.6340
Epoch 66/300
1619/1619 [==============================] - 8s 5ms/step - loss: 0.7387 - acc: 0.6881 - val_loss: 0.7897 - val_acc: 0.6441
Epoch 67/300
1619/1619 [========================

1619/1619 [==============================] - 5s 3ms/step - loss: 0.5817 - acc: 0.7603 - val_loss: 0.7502 - val_acc: 0.6816
Epoch 121/300
1619/1619 [==============================] - 5s 3ms/step - loss: 0.5746 - acc: 0.7622 - val_loss: 0.7267 - val_acc: 0.6960
Epoch 122/300
1619/1619 [==============================] - 5s 3ms/step - loss: 0.5775 - acc: 0.7634 - val_loss: 0.8023 - val_acc: 0.6499
Epoch 123/300
1619/1619 [==============================] - 5s 3ms/step - loss: 0.5701 - acc: 0.7647 - val_loss: 0.7659 - val_acc: 0.6700
Model architecture saved to: /home/moorea/tdparse/model zoo/TDLSTM SemEval 14 Laptop architecture.yaml
Model weights saved to /home/moorea/tdparse/model zoo/TDLSTM SemEval 14 Laptop weights.h5
Save time 1.56


/home/moorea/.virtualenvs/tdparse/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/moorea/.virtualenvs/tdparse/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/moorea/.virtualenvs/tdparse/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


saving raw data
SemEval 14 Restaurant {'tokeniser': <function ark_twokenize at 0x7f85f53060d0>, 'embeddings': glove twitter 200d, 'lower': True, 'pad_size': 9}
Train on 2521 samples, validate on 1081 samples
Epoch 1/300
2521/2521 [==============================] - 10s 4ms/step - loss: 1.0037 - acc: 0.5954 - val_loss: 0.9658 - val_acc: 0.6004
Epoch 2/300
2521/2521 [==============================] - 11s 4ms/step - loss: 0.9612 - acc: 0.6010 - val_loss: 0.9577 - val_acc: 0.6004
Epoch 3/300
2521/2521 [==============================] - 11s 4ms/step - loss: 0.9559 - acc: 0.6010 - val_loss: 0.9541 - val_acc: 0.6004
Epoch 4/300
2521/2521 [==============================] - 11s 4ms/step - loss: 0.9515 - acc: 0.6010 - val_loss: 0.9507 - val_acc: 0.6004
Epoch 5/300
2521/2521 [==============================] - 10s 4ms/step - loss: 0.9476 - acc: 0.6010 - val_loss: 0.9473 - val_acc: 0.6004
Epoch 6/300
2521/2521 [==============================] - 11s 4ms/step - loss: 0.9434 - acc: 0.6010 - val_loss: 0

...

538/538 [==============================] - 5s 9ms/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 124/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 125/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 126/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0033 - val_acc: 1.0000
Epoch 127/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0033 - val_acc: 1.0000
Epoch 128/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0033 - val_acc: 1.0000
Epoch 129/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0033 - val_acc: 1.0000
Epoch 130/300
538/538 [==============================] - 5s 9ms/step 

Epoch 184/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 185/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 186/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 187/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 188/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 189/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 190/300
538/538 [==============================] - 5s 9ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 191/300
538/538 [==============================] 

390/390 [==============================] - 6s 14ms/step - loss: 1.0586 - acc: 0.6077 - val_loss: 1.0030 - val_acc: 0.6310
Epoch 2/300
390/390 [==============================] - 4s 11ms/step - loss: 0.9874 - acc: 0.6308 - val_loss: 0.9467 - val_acc: 0.6310
Epoch 3/300
390/390 [==============================] - 4s 11ms/step - loss: 0.9448 - acc: 0.6308 - val_loss: 0.9131 - val_acc: 0.6310
Epoch 4/300
390/390 [==============================] - 4s 11ms/step - loss: 0.9194 - acc: 0.6308 - val_loss: 0.8929 - val_acc: 0.6310
Epoch 5/300
390/390 [==============================] - 4s 11ms/step - loss: 0.9040 - acc: 0.6308 - val_loss: 0.8812 - val_acc: 0.6310
Epoch 6/300
390/390 [==============================] - 4s 11ms/step - loss: 0.8946 - acc: 0.6308 - val_loss: 0.8760 - val_acc: 0.6310
Epoch 7/300
390/390 [==============================] - 4s 11ms/step - loss: 0.8903 - acc: 0.6308 - val_loss: 0.8708 - val_acc: 0.6310
Epoch 8/300
390/390 [==============================] - 4s 11ms/step - loss

Epoch 62/300
390/390 [==============================] - 4s 11ms/step - loss: 0.7919 - acc: 0.6385 - val_loss: 0.8253 - val_acc: 0.6310
Epoch 63/300
390/390 [==============================] - 4s 11ms/step - loss: 0.7898 - acc: 0.6385 - val_loss: 0.8244 - val_acc: 0.6310
Epoch 64/300
390/390 [==============================] - 4s 11ms/step - loss: 0.7872 - acc: 0.6359 - val_loss: 0.8233 - val_acc: 0.6310
Epoch 65/300
390/390 [==============================] - 4s 11ms/step - loss: 0.7855 - acc: 0.6359 - val_loss: 0.8224 - val_acc: 0.6310
Epoch 66/300
390/390 [==============================] - 4s 11ms/step - loss: 0.7831 - acc: 0.6333 - val_loss: 0.8216 - val_acc: 0.6310
Epoch 67/300
390/390 [==============================] - 2s 4ms/step - loss: 0.7805 - acc: 0.6385 - val_loss: 0.8225 - val_acc: 0.6369
Epoch 68/300
390/390 [==============================] - 2s 4ms/step - loss: 0.7781 - acc: 0.6410 - val_loss: 0.8246 - val_acc: 0.6369
Epoch 69/300
390/390 [==============================] - 4

Epoch 123/300
390/390 [==============================] - 2s 4ms/step - loss: 0.5961 - acc: 0.7667 - val_loss: 0.8373 - val_acc: 0.6607
Epoch 124/300
390/390 [==============================] - 2s 4ms/step - loss: 0.6026 - acc: 0.7564 - val_loss: 0.7882 - val_acc: 0.6190
Epoch 125/300
390/390 [==============================] - 4s 11ms/step - loss: 0.5818 - acc: 0.7718 - val_loss: 0.7571 - val_acc: 0.6548
Epoch 126/300
390/390 [==============================] - 2s 4ms/step - loss: 0.5932 - acc: 0.7718 - val_loss: 0.8378 - val_acc: 0.5893
Epoch 127/300
390/390 [==============================] - 2s 4ms/step - loss: 0.5943 - acc: 0.7744 - val_loss: 0.7784 - val_acc: 0.6310
Epoch 128/300
390/390 [==============================] - 2s 4ms/step - loss: 0.5810 - acc: 0.7718 - val_loss: 0.7780 - val_acc: 0.6250
Epoch 129/300
390/390 [==============================] - 2s 4ms/step - loss: 0.5770 - acc: 0.7744 - val_loss: 0.7602 - val_acc: 0.6488
Epoch 130/300
390/390 [==============================]

# Mass Evaluation of the TCLSTM model

In [ ]:
for dataset_name, train_test in dataset_train_test.items():
    train, test = train_test
    model_params = {'tokeniser' : ark_twokenize, 'embeddings' : glove_twit_200,
                    'lower' : True, 'pad_size' : half_average_dataset(train)}
    dataset_predictions(train, test, dataset_name, TCLSTM, model_params)